In [1]:
DTYPES = {
    'date_time': 'string',
    'site_name': 'uint8',
    'posa_continent': 'uint8',
    'user_location_country': 'uint8',
    'user_location_region': 'uint16',
    'user_location_city': 'uint16',
    'orig_destination_distance': 'float32',
    'user_id': 'uint32',
    'is_mobile': 'bool',
    'is_package': 'bool',
    'channel': 'uint8',
    'srch_ci': 'string',
    'srch_co': 'string',
    'srch_adults_cnt': 'uint8',
    'srch_children_cnt': 'uint8',
    'srch_rm_cnt': 'uint8',
    'srch_destination_id': 'uint16',
    'srch_destination_type_id': 'uint8',
    'is_booking': 'bool',
    'cnt': 'uint16',
    'hotel_continent': 'uint8',
    'hotel_country': 'uint8',
    'hotel_market': 'uint16',
    'hotel_cluster': 'uint8',
}
DATETIME_COLUMNS = ['date_time', 'srch_ci', 'srch_co']
BOOLEAN_COLUMNS = ['is_booking', 'is_mobile', 'is_package']

In [2]:
import numpy as np
import polars as pl


def map_to_polars(dtype: str):
    conversion = {
        'string': pl.String,
        'uint8': pl.UInt8,
        'uint16': pl.UInt16,
        'uint32': pl.UInt32,
        'int16': pl.Int16,
        'int32': pl.Int32,
        'float32': pl.Float32,
        'bool': pl.UInt8 
    }
    return conversion[dtype]

def map_to_np(dtype):
    conversion = {
        pl.String: 'string',
        pl.UInt8: 'uint8',
        pl.UInt16: 'uint16',
        pl.UInt32: 'uint32',
        pl.Float32: 'float32',
        pl.Int8: 'int8',
        pl.Int16: 'int16',
        pl.Int32: 'int32',
        pl.Boolean: 'bool'
    }
    return conversion.get(dtype, 'datetime')

In [3]:
dtypes = {k: map_to_polars(v) for k, v in DTYPES.items()}
df = pl.read_csv('../data/raw/train.csv', dtypes=dtypes)
df = df.with_columns(
    *[pl.col(col).str.to_datetime() for col in DATETIME_COLUMNS]
)
for col in BOOLEAN_COLUMNS:
    df = df.replace(col, df[col] == 1)

C:\Users\tyoma\AppData\Local\Temp\ipykernel_2796\93873269.py:7: DeprecationWarning: `replace` is deprecated. DataFrame.replace is deprecated and will be removed in a future version. Please use
    df = df.with_columns(new_column.alias(column_name))
instead.
  df = df.replace(col, df[col] == 1)


In [4]:
TARGET = 'is_booking'

In [6]:
from collections.abc import Callable
from dataclasses import dataclass, field


@dataclass
class DataPreparationPipelineState:
    schema: dict[str, str] = field(default_factory=dict)
    numerical_columns: list[str] = field(default_factory=list)
    categorical_columns: list[str] = field(default_factory=list)
    numerical_categorical_columns: list[str] = field(default_factory=list)


class DataPreparationPipeline:
    '''
    Lazy pipeline for data preprocessing
    '''
    Task = Callable[[tuple[pl.DataFrame, DataPreparationPipelineState]], tuple[pl.DataFrame, DataPreparationPipelineState]]
    def __init__(self) -> None:
        self.pipeline = []

    def _add_task(self, task: Task) -> None:
        self.pipeline.append(task)

    def add_columns(self, expressions: dict[pl.expr, str]) -> 'FeatureBuilder':
        '''
        Add column to a DataFrame
        :param expressions: dict with keys as column names, values are tuples of polars expressions and result dtype
        '''

        def task(df: pl.DataFrame, state: DataPreparationPipelineState) -> tuple[pl.DataFrame, DataPreparationPipelineState]:
            df = df.with_columns(**{
                col: expr.cast(map_to_polars(dtype)) for col, (expr, dtype) in expressions.items() 
            })
            for col, (_, dtype) in expressions.items():
                state.schema[col] = dtype
            return df, state
        
        self._add_task(task)
        return self
    
    def map_columns(self, expressions: dict[pl.expr, str]) -> 'FeatureBuilder':
        '''
        Maps column in a DataFrame
        :param expressions: dict with keys as column names, values are tuples of polars expressions and result dtype
        '''

        return self.add_columns(expressions)

    def drop_columns(self, columns: list[str]) -> 'FeatureBuilder':
        '''
        Drops columns from dataframe
        :param columns: columns to drop
        '''

        def task(df: pl.DataFrame, state: DataPreparationPipelineState) -> tuple[pl.DataFrame, DataPreparationPipelineState]:
            df = df.drop(*columns)
            state.schema = {k: v for k, v in state.schema.items() if k not in columns}
            return df, state
        
        self._add_task(task)
        return self
    
    def cast(self, to: dict[str, str]) -> 'FeatureBuilder':
        '''
        Casts given columns to given dtype
        :param to: dict[column, dtype]
        '''

        def task(df: pl.DataFrame, state: DataPreparationPipelineState) -> tuple[pl.DataFrame, DataPreparationPipelineState]:
            df = df.with_columns(**{
                col: pl.col(col).cast(map_to_polars(dtype)) for col, dtype in to.items()   
            })
            for col, dtype in to.items():
                state.schema[col] = dtype
            return df, state
        
        self._add_task(task)
        return self
    
    def make_column_split(self, num_cat_threshold: int = 0) -> 'FeatureBuilder':
        '''
        Splits columns into 3 categories:
        1. Categorical columns
        2. "Numerical categorical" columns (numerical columns where number of unique values less than threshold)
        3. Numerical columns

        :param num_cat_threshold: number of unique values, below which numerical feature is considered as numerical categorical
        by default it is 0, so there are no numerical categorical columns
        '''

        def task(df: pl.DataFrame, state: DataPreparationPipelineState) -> tuple[pl.DataFrame, DataPreparationPipelineState]:
            state.numerical_categorical_columns = [
                col for col in df.columns
                if df[col].dtype not in [pl.Boolean, pl.String, pl.Datetime] and df[col].n_unique() < num_cat_threshold
            ]
            state.numerical_columns = [
                col for col in df.columns
                if df[col].dtype not in [pl.Boolean, pl.String, pl.Datetime] and col not in state.numerical_categorical_columns
            ]
            state.categorical_columns = [
                col for col in df.columns
                if col not in state.numerical_columns and col not in state.numerical_categorical_columns
            ]
            return df, state

        self._add_task(task)
        return self
    
    def __call__(self, df: pl.DataFrame) -> tuple[pl.DataFrame, DataPreparationPipelineState]:
        '''
        Processes the DataFrame
        :param df: pl.DataFrame to process
        :returns: tuple of processed DataFrame and last pipeline state 
        '''
        
        state = DataPreparationPipelineState(
            schema={
                k: map_to_np(v) for k, v in df.schema.items()
            }
        )
        for task in self.pipeline:
            df, state = task(df, state)
        return df, state

In [31]:
feature_extractor = DataPreparationPipeline() \
    .add_columns({
        'co_ci_diff': ((pl.col('srch_co') - pl.col('srch_ci')).dt.total_days(), 'int16'),
        'ci_dt_diff': ((pl.col('srch_ci') - pl.col('date_time')).dt.total_days(), 'int32'),
        'ci_weekday': (pl.col('srch_ci').dt.weekday(), 'uint8'),
        'co_weekday': (pl.col('srch_co').dt.weekday(), 'uint8'),
        'date_time_weekday': (pl.col('date_time').dt.weekday(), 'uint8')
    }) \
    .drop_columns(['orig_destination_distance', 'srch_ci', 'srch_co']) \
    .cast({
        'is_mobile': 'uint8',
        'is_package': 'uint8'
    }) \
    .make_column_split(num_cat_threshold=250)

In [32]:
data, state = feature_extractor(df)

ValueError: too many values to unpack (expected 2)

In [9]:
x = data.drop(TARGET)
y = data[TARGET]

In [10]:
from collections.abc import Callable

from numpy.typing import NDArray
from sklearn.base import BaseEstimator


Metric = Callable[[pl.DataFrame, pl.DataFrame, BaseEstimator], float]

In [11]:
from datetime import datetime, timedelta
from math import floor

import pandas as pd

from sklearn.base import BaseEstimator, clone
from tqdm.notebook import tqdm


def get_interval(start: datetime, end: datetime, dt_column: pl.Series) -> pl.Series:
    return (dt_column >= start) & (dt_column < end)

def handle_nans(df: pd.DataFrame) -> pd.DataFrame:
    df[df.isna()] = np.nan
    return df


def blocked_cross_validation(
    start: datetime,
    end: datetime,
    training_interval_len: timedelta,
    test_interval_len: timedelta,
    estimator: BaseEstimator,
    x: pl.DataFrame,
    y: pl.DataFrame,
    dt_column: pl.Series,
    metrics: dict[str, Metric],
) -> tuple[dict[str, list[float]], list[BaseEstimator]]:
    result_metrics = {key: [] for key in metrics.keys()}
    estimators = []
    n_intervals = floor((end - test_interval_len - start) / training_interval_len)
    for i in tqdm(range(n_intervals - 1)):
        est = clone(estimator)
        training_dt_interval = start + i * training_interval_len, start + (i + 1) * training_interval_len
        test_dt_interval = training_dt_interval[-1], training_dt_interval[-1] + test_interval_len
        training_interval = get_interval(*training_dt_interval, dt_column)
        test_interval = get_interval(*test_dt_interval, dt_column)
        est = est.fit(
            handle_nans(x.filter(training_interval).to_pandas()), 
            y.filter(training_interval).to_pandas()
        )
        x_test = x.filter(test_interval)
        y_test = y.filter(test_interval)

        for key, metric in metrics.items():
            result_metrics[key].append(metric(x_test, y_test, est))
        
        estimators.append(est)

    return result_metrics, estimators        

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


cat_columns = state.categorical_columns.copy()
cat_columns.remove('is_booking')
cat_columns.remove('date_time')

numerical_transform = Pipeline([
    ('NumericalImputer', SimpleImputer(strategy='median', missing_values=np.nan)),
    ('Scaler', StandardScaler()),
])

cat_transform = Pipeline([
    ('CategoricalImputer', SimpleImputer(strategy='most_frequent')),
    ('OneHot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_oh_transform = Pipeline([
    ('Imputer', SimpleImputer(strategy='most_frequent')),
    ('OneHot', OneHotEncoder(handle_unknown='ignore'))
])


base_pipeline = Pipeline([
    (
        'TransformingColumns',
        ColumnTransformer([
            ('Numerical', numerical_transform, state.numerical_columns),
            ('Categorical', cat_transform, cat_columns),
            ('NumCategorical', numerical_oh_transform, state.numerical_categorical_columns)
        ])
    ),
    ('Logreg', LogisticRegression(max_iter=1_000))
])

In [26]:
from collections import defaultdict

from numpy.typing import NDArray


class DataSampler:
    '''
    Class responsible for sampling negative samples for validation
    '''

    def __init__(self, df: pl.DataFrame) -> None:
        cluster_country = df.select('hotel_country', 'hotel_market', 'hotel_cluster')
        unique_values = cluster_country.unique()
        cluster_to_hotel_market = defaultdict(list)
        cluster_to_hotel_country = defaultdict(list)

        for row in unique_values.iter_rows():
            cluster_to_hotel_country[row[2]].append(row[0])
            cluster_to_hotel_market[row[2]].append(row[1])

        self.hotel_data = {
            key: (np.array(cluster_to_hotel_market[key]), np.array(cluster_to_hotel_country[key]))
            for key in cluster_to_hotel_country.keys()
        }
        self.columns = df.columns
        self.country_idx = self.columns.index('hotel_country')
        self.market_idx = self.columns.index('hotel_market')
        self.cluster_idx = self.columns.index('hotel_cluster')
    
    def sample(self, row: tuple, n: int) -> NDArray:
        cluster = row[self.cluster_idx]
        markets, countries = self.hotel_data[cluster]
        indices = np.random.choice(len(markets), size=n)
        row = np.array(row)[None, :].repeat(n + 1, 0)
        row[1:, self.market_idx] = markets[indices]
        row[1:, self.country_idx] = countries[indices]
        return row

In [27]:
from collections.abc import Generator

from sklearn.metrics import accuracy_score


def accuracy(x_test: NDArray, y_test: NDArray, estimator: BaseEstimator) -> float:
    return accuracy_score(y_test, estimator.predict(x_test))


def iterate_batched(data: pl.DataFrame, batch_size: int) -> Generator[pl.DataFrame, None, None]:
    i = 0
    j = min(batch_size, len(data))
    yield data[i:j]
    while j < len(data):
        i += batch_size
        j = min(j + batch_size, len(data))
        yield data[i:j]


def batch_mrr(x_test: pl.DataFrame, estimator: BaseEstimator, sampler: DataSampler, n: int = 5) -> float:
    rows = np.row_stack([
        sampler.sample(row, n)
        for row in x_test.iter_rows()
    ])
    rows = pl.DataFrame(
        data={
            col: rows[:, idx].astype(map_to_np(x_test[col].dtype)) for idx, col in enumerate(x_test.columns)
        },
        schema=x_test.schema
    )
    predictions = estimator.predict_proba(rows.to_pandas())[:, 1]
    predictions = predictions.reshape(len(x_test), n + 1)
    real_values = predictions[:, 0]
    predictions = np.sort(predictions, axis=1)[:, ::-1]
    indices = (predictions == real_values.reshape(-1, 1)).argmax(axis=1)
    return np.sum(1 / (1 + indices))


def mrr(x_test: pl.DataFrame, y_test: pl.DataFrame, 
        estimator: BaseEstimator, sampler: DataSampler, 
        n: int = 5, batch_size: int = 1024) -> float:
    x_test = x_test.filter(y_test)
    mrr_ = 0
    for batch in iterate_batched(x_test, batch_size):
        mrr_ += batch_mrr(batch, estimator, sampler, n)
    return mrr_ / len(x_test)


In [28]:
from functools import partial


metrics, estimators = blocked_cross_validation(
    x['date_time'].min(),
    x['date_time'].max(),
    timedelta(days=14),
    timedelta(days=7),
    base_pipeline,
    x=x.drop('date_time'),
    y=y,
    dt_column=x['date_time'],
    metrics={
        'mrr': partial(mrr, sampler=DataSampler(x.drop('date_time')), n=20, batch_size=1024),
        'accuracy': accuracy
    }
)

  0%|          | 0/50 [00:00<?, ?it/s]

ValueError: SimpleImputer does not support data with dtype bool. Please provide either a numeric array (with a floating point or integer dtype) or categorical data represented either as an array with integer dtype or an array of string values with an object dtype.